# An improved version of Deep Reinforcement Learning

(Original Notebook can be found [here.](https://www.kaggle.com/alexisbcook/deep-reinforcement-learning))


Improvements done: 
- Added One Step Look Ahead (OSLA) and Alpha - Beta Pruned Three Step Look Ahead (TSLA) Agents.
- Assigned 3 different agents as training partners for RL agent Training



In [ ]:
#First, required libraries are imported:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Stable Baselines is not compatible with TensorFlow 2.0, thus, downgrading is required.
!pip install 'tensorflow==1.15.0'

In [ ]:
# Checking the current version of tensorflow. It should be "1.15.0"
import tensorflow as tf
tf.__version__

# Creating an alpha-beta pruned game-playing agent

Firstly, defining helper functions that are provided in the previous modules. 

In [ ]:
# Helper function for get_heuristic: checks if window satisfies heuristic conditions
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

In [ ]:
# Helper function for minimax: calculates value of heuristic for grid
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    num_fours_opp = count_windows(grid, 4, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
    return score

In [ ]:
# Uses minimax to calculate value of dropping piece in selected column
def score_move(grid, col, mark, config, nsteps):
    next_grid = drop_piece(grid, col, mark, config)
    score = minimax(next_grid, nsteps-1, False, mark, config)
    return score

# Helper function for minimax: checks if agent or opponent has four in a row in the window
def is_terminal_window(window, config):
    return window.count(1) == config.inarow or window.count(2) == config.inarow

# Helper function for minimax: checks if game has ended
def is_terminal_node(grid, config):
    # Check for draw 
    if list(grid[0, :]).count(0) == 0:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal 
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if is_terminal_window(window, config):
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if is_terminal_window(window, config):
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    return False



In [ ]:
# Minimax implementation
def minimax(node, depth, maximizingPlayer, mark, config, alpha = -np.Inf, beta = np.Inf):
    is_terminal = is_terminal_node(node, config)
    valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
    if depth == 0 or is_terminal:
        return get_heuristic(node, mark, config)
    if maximizingPlayer:
        value = -np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            value = max(value, minimax(child, depth-1, False, mark, config, alpha, beta))
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value
    else:
        value = np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark%2+1, config)
            value = min(value, minimax(child, depth-1, True, mark, config, alpha, beta))
            beta = min(beta, value)
            if beta <= alpha:
                break                
        return value

In [ ]:
# Depthness of the game tree. 
N_STEPS = 3

# Defining the Alpha Pruned 3 Step Look Ahead (TSLA) Agent. 
def agentTSLA(obs, config):
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

In [ ]:
N_STEPS = 1

# Defining the One Step Look Ahead (OSLA) Agent.
def agentOSLA(obs, config):
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

As the agent defined above, let's check if it works against a random agent:

In [ ]:
#This cell runs a FSLA agent against a random agent. 
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx")

# Run TSLA agent vs a random agent
env.run([agentTSLA, "random"])

# Run TSLA agent vs OSLA agent
#env.run([agentTSLA, agentOSLA])


# Show the game
env.render(mode="ipython")

Let's define a function to check which decision making algorithm is more likely to win. Random or FSLA?

In [ ]:
#This function calculates the percentage of wins per "n" rounds against the other agent given in the parameter. 
def get_win_percentages(agent1, agent2, n_rounds=500):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent1, agent2], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
get_win_percentages(agent1=agentTSLA, agent2=agentOSLA, n_rounds=100)

As it is observed, an TSLA agent is more likely to win against a random agent.
Now, let's train a Reinforcement Learning Agent using the newly defined TSLA agent.

# Creating an Open AI Gym environment

`ConnectFourGym` class implements ConnectX as an [OpenAI Gym environment](http://gym.openai.com/docs/) and uses several methods:
- `reset()` will be called at the beginning of every game.  It returns the starting game board as a 2D numpy array with 6 rows and 7 columns.
- `change_reward()` customizes the rewards that the agent receives.  (_The competition already has its own system for rewards that are used to rank the agents, and this method changes the values to match the rewards system we designed._) 
- `step()` is used to play the agent's choice of action (supplied as `action`), along with the opponent's response.  It returns:
  - the resulting game board (as a numpy array), 
  - the agent's reward (from the most recent move only: one of `+1`, `-10`, `-1`, or `1/42`), and
  - whether or not the game has ended (if the game has ended, `done=True`; otherwise, `done=False`).

In [ ]:
from kaggle_environments import make, evaluate
from gym import spaces

class ConnectFourGym:
    def __init__(self, agent2="random"):
        ks_env = make("connectx", debug=True)
        self.env = ks_env.train([None, agent2])
        self.rows = ks_env.configuration.rows
        self.columns = ks_env.configuration.columns
        # Learn about spaces here: http://gym.openai.com/docs/#spaces
        self.action_space = spaces.Discrete(self.columns)
        self.observation_space = spaces.Box(low=0, high=2, 
                                            shape=(self.rows,self.columns,1), dtype=np.int)
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None
    def reset(self):
        self.obs = self.env.reset()
        return np.array(self.obs['board']).reshape(self.rows,self.columns,1)
    def change_reward(self, old_reward, done):
        if old_reward == 1: # The agent won the game
            return 1
        elif done: # The opponent won the game
            return -1
        else: # Reward 1/42
            return 1/(self.rows*self.columns)
    def step(self, action):
        # Check if agent's move is valid
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid: # Play the move
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else: # End the game and penalize agent
            reward, done, _ = -10, True, {}
        return np.array(self.obs['board']).reshape(self.rows,self.columns,1), reward, done, _

Let's train an agent to beat the FSLA agent:

In [ ]:
# Create ConnectFour environment
env = ConnectFourGym(agent2=agentOSLA)

# env = ConnectFourGym(agent2=agentTSLA)
# env = ConnectFourGym(agent2="random")


[Stable Baselines](https://github.com/hill-a/stable-baselines) is used for the implementation of reinforcement learning algorithm.

It is a set of improved implementations of reinforcement learning algorithms based on OpenAI Baselines.

Stable Baselines requires us to work with ["vectorized" environments](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html).  For this, we can use the `DummyVecEnv` class.  

The `Monitor` class lets us watch how the agent's performance gradually improves, as it plays more and more games.

In [ ]:
!apt-get update
!apt-get install -y cmake libopenmpi-dev python3-dev zlib1g-dev
!pip install "stable-baselines[mpi]==2.9.0"

In [ ]:
import os
from stable_baselines.bench import Monitor 
from stable_baselines.common.vec_env import DummyVecEnv

# Create directory for logging training information
log_dir = "ppo/"
os.makedirs(log_dir, exist_ok=True)

# Logging progress
monitor_env = Monitor(env, log_dir, allow_early_resets=True)

# Create a vectorized environment
vec_env = DummyVecEnv([lambda: monitor_env])

The next step is to specify the architecture of the neural network.  In this case, we use a convolutional neural network.  To learn more about how to specify architectures with Stable Baselines, check out the documentation [here](https://stable-baselines.readthedocs.io/en/master/guide/custom_policy.html).

Note that this is the neural network that outputs the probabilities of selecting each column.  Since we use the PPO algorithm (`PPO1` in the code cell below), our network will also output some additional information (called the "value" of the input).  This is outside the scope of this course, but you can learn more by reading about "actor-critic networks".

In [ ]:
from stable_baselines import PPO1 
from stable_baselines.a2c.utils import conv, linear, conv_to_fc
from stable_baselines.common.policies import CnnPolicy

# Neural network for predicting action values
def modified_cnn(scaled_images, **kwargs):
    activ = tf.nn.relu
    layer_1 = activ(conv(scaled_images, 'c1', n_filters=32, filter_size=3, stride=1, 
                         init_scale=np.sqrt(2), **kwargs))
    layer_2 = activ(conv(layer_1, 'c2', n_filters=64, filter_size=3, stride=1, 
                         init_scale=np.sqrt(2), **kwargs))
    layer_2 = conv_to_fc(layer_2)
    return activ(linear(layer_2, 'fc1', n_hidden=512, init_scale=np.sqrt(2)))  

class CustomCnnPolicy(CnnPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomCnnPolicy, self).__init__(*args, **kwargs, cnn_extractor=modified_cnn)
        
# Initialize agent
model = PPO1(CustomCnnPolicy, vec_env, verbose=0)

In the code cell above, the weights of the neural network are initially set to random values.

In the next code cell, we "train the agent", which is just another way of saying that we find weights of the neural network that are likely to result in the agent selecting good moves.

We plot a rolling average of the cumulative reward that the agent received during training.  As evidenced by the increasing function, the agent gradually learned to perform better by playing the game.

In [ ]:
# Train agent
model.learn(total_timesteps=100000)

# Plot cumulative reward
with open(os.path.join(log_dir, "monitor.csv"), 'rt') as fh:    
    firstline = fh.readline()
    assert firstline[0] == '#'
    df = pd.read_csv(fh, index_col=None)['r']
df.rolling(window=1000).mean().plot()
plt.show()

In [ ]:
def agentRL(obs, config):
    # Use the best model to select a column
    col, _ = model.predict(np.array(obs['board']).reshape(6,7,1))
    # Check if selected column is valid
    is_valid = (obs['board'][int(col)] == 0)
    # If not valid, select random move. 
    if is_valid:
        return int(col)
    else:
        return random.choice([col for col in range(config.columns) if obs.board[int(col)] == 0])

In the next code cell, we see the outcome of one game round against a random agent.

In [ ]:
# Create the game environment
env = make("connectx")

# Two random agents play one game round
env.run([agentRL, "random"])
#env.run([RLAgent, oslaAgent])
#env.run([oslaAgent, "random"])

# Show the game
env.render(mode="ipython")

And, we calculate how it performs on average, against the random agent.

In [ ]:
get_win_percentages(agent1 = agentRL, agent2 = agentOSLA, n_rounds = 1000)
get_win_percentages(agent1 = agentRL, agent2 = agentTSLA, n_rounds = 1000)
get_win_percentages(agent1 = agentRL, agent2 = "random", n_rounds = 1000)
